In [1]:
!pip -q install -U keras-tuner gputil

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 3.3 MB/s eta 0:00:00


In [2]:
import os, json, time, platform, zipfile
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import keras_tuner as kt
import GPUtil
from google.colab import files

In [3]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)


Mounted at /content/drive


In [4]:
# Point these folders to where each window's .npy live
WINDOW_DIRS = {
    "48to12": "/content/drive/MyDrive/EdgeMeter_AIv2/data",
    "96to12": "/content/drive/MyDrive/EdgeMeter_AIv2/data",
}
# Map window keys to filename suffixes
WINDOW_TAGS = {
    "48to12": "48_12",
    "96to12": "96_12",
}

# Reproducibility
tf.random.set_seed(42)
np.random.seed(42)

In [5]:
# Loader that enforces the correct window tag
def _load_for_tag(d, base, tag):
    """Load base.npy or base_<tag>.npy; prefer exact tag when present."""
    p_tag = os.path.join(d, f"{base}_{tag}.npy")
    p_plain = os.path.join(d, f"{base}.npy")
    if os.path.exists(p_tag):
        return np.load(p_tag)
    if os.path.exists(p_plain):
        return np.load(p_plain)
    raise FileNotFoundError(f"Missing {base}_{tag}.npy (or {base}.npy) in {d}")

def load_split_npys(d, tag):
    X_train = _load_for_tag(d, "X_train", tag)
    y_train = _load_for_tag(d, "y_train", tag)
    X_val   = _load_for_tag(d, "X_val",   tag)
    y_val   = _load_for_tag(d, "y_val",   tag)
    X_test  = _load_for_tag(d, "X_test",  tag)
    y_test  = _load_for_tag(d, "y_test",  tag)

    # V2 shapes: X=(N,T,F), y=(N,12)
    assert X_train.ndim==3 and X_val.ndim==3 and X_test.ndim==3, "X must be (N,T,F)"
    assert y_train.ndim==2 and y_val.ndim==2 and y_test.ndim==2, "y must be (N,12)"
    assert y_train.shape[1]==12 and y_val.shape[1]==12 and y_test.shape[1]==12, "y must have 12 steps"

    # Hard NaN checks
    for arr, name in [(X_train,"X_train"),(X_val,"X_val"),(X_test,"X_test"),
                      (y_train,"y_train"),(y_val,"y_val"),(y_test,"y_test")]:
        if np.isnan(arr).any():
            raise ValueError(f"NaNs detected in {name}")

    return X_train, y_train, X_val, y_val, X_test, y_test


In [6]:
# Model builder
def make_builder(timesteps, n_features, out_steps):
    def build_model(hp):
        units   = hp.Int('units', min_value=32, max_value=128, step=32)
        dropout = hp.Float('dropout', min_value=0.0, max_value=0.5, step=0.1)
        lr      = hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])

        model = keras.Sequential([
            layers.Input(shape=(timesteps, n_features)),
            layers.LSTM(units),
            layers.Dropout(dropout),
            layers.Dense(out_steps)
        ])
        model.compile(
            optimizer=keras.optimizers.Adam(learning_rate=lr),
            loss='mse',
            metrics=['mae']
        )
        return model
    return build_model

In [7]:
# GPU info
gpus = tf.config.list_physical_devices('GPU')
print("GPUs visible to TF:", gpus)
if gpus:
    try:
        tf.config.experimental.set_memory_growth(tf.config.list_physical_devices('GPU')[0], True)
    except:
        pass
gpu_name = GPUtil.getGPUs()[0].name if GPUtil.getGPUs() else "None"
print("GPU in use:", gpu_name)

GPUs visible to TF: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
GPU in use: Tesla T4


In [1]:
for win_key, DATA_DIR in WINDOW_DIRS.items():
    tag = WINDOW_TAGS[win_key]
    print(f"\n Tuning window: {win_key} (tag {tag}) | dir: {DATA_DIR} ")
    LOG_PATH = os.path.join(DATA_DIR, f"lstm_tuning_log_{win_key}.json")
    HP_PATH  = os.path.join(DATA_DIR, f"best_lstm_hp_{win_key}.json")

    # Load correct files for this window
    X_train, y_train, X_val, y_val, X_test, y_test = load_split_npys(DATA_DIR, tag)
    print(f"Train X: {X_train.shape} | y: {y_train.shape}")
    print(f"Val   X: {X_val.shape}   | y: {y_val.shape}")
    print(f"Test  X: {X_test.shape}  | y: {y_test.shape}")

    # sanity: enforce expected timesteps per window
    expected_T = 48 if win_key == "48to12" else 96
    assert X_train.shape[1] == expected_T, f"Loaded wrong window: expected T={expected_T}, got {X_train.shape[1]}"

    timesteps  = X_train.shape[1]
    n_features = X_train.shape[2]
    out_steps  = y_train.shape[1]

    # 5% subset (same as V1)
    subset_frac = 0.05
    X_train_small = X_train[:int(subset_frac * X_train.shape[0])]
    y_train_small = y_train[:int(subset_frac * y_train.shape[0])]
    X_val_small   = X_val[:int(subset_frac * X_val.shape[0])]
    y_val_small   = y_val[:int(subset_frac * y_val.shape[0])]
    print(f"Subset shapes: {X_train_small.shape} | {X_val_small.shape}")

    # Builder + Tuner
    builder = make_builder(timesteps, n_features, out_steps)
    tuner = kt.BayesianOptimization(
        builder,
        objective='val_loss',
        max_trials=20,
        directory=DATA_DIR,
        project_name=f'lstm_retune_{win_key}'   # unique per window
    )

    early_stop = keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=4,
        restore_best_weights=True
    )

    # Search
    start_time = time.time()
    with tf.device('/GPU:0'):
        tuner.search(
            X_train_small, y_train_small,
            validation_data=(X_val_small, y_val_small),
            epochs=50,
            batch_size=512,
            callbacks=[early_stop],
            verbose=1
        )
    end_time = time.time()

    # Saving best HPs
    best_hp = tuner.get_best_hyperparameters(1)[0]
    hp_dict = {
        'units': best_hp.get('units'),
        'dropout': best_hp.get('dropout'),
        'learning_rate': best_hp.get('learning_rate')
    }
    with open(HP_PATH, 'w') as f:
        json.dump(hp_dict, f, indent=4)

    # Log
    model_tmp = builder(best_hp)
    total_params = model_tmp.count_params()
    platform_info = platform.platform()

    log = {
        "window": win_key,
        "model": "LSTM",
        "task": "Smart Meter Energy Forecasting",
        "tuning_type": "BayesianOptimization",
        "tuning_time_minutes": round((end_time - start_time) / 60, 2),
        "best_hyperparameters": hp_dict,
        "total_params": int(total_params),
        "input_shape": list(X_train.shape[1:]),
        "sequence_length": int(X_train.shape[1]),
        "gpu_used": gpu_name,
        "platform": platform_info,
        "log_type": "Tuning"
    }
    with open(LOG_PATH, 'w') as f:
        json.dump(log, f, indent=4)

    print(f"[{win_key}] Best HPs saved → {HP_PATH}")
    print(f"[{win_key}] Tuning log  → {LOG_PATH}")

print("\nAll tuning runs completed.")

NameError: name 'WINDOW_DIRS' is not defined

In [2]:
# -*- coding: utf-8 -*-
# LSTM Hyperparameter Tuning — 96→12 only (Colab + Google Drive)
# Same model/tuner as V1; 5% subset; per-window logs; memmap I/O to avoid RAM spikes.

!pip -q install -U keras-tuner gputil

import os, json, time, platform
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import keras_tuner as kt
import GPUtil
from google.colab import drive

# ---- Mount Drive ----
drive.mount('/content/drive', force_remount=True)

# ---- Point to your .npy folder (same folder is fine for both windows) ----
DATA_DIR = "/content/drive/MyDrive/EdgeMeter_AIv2/data"  # <-- change if yours differs
WIN_KEY  = "96to12"
WIN_TAG  = "96_12"

HP_PATH  = os.path.join(DATA_DIR, f"best_lstm_hp_{WIN_KEY}.json")
LOG_PATH = os.path.join(DATA_DIR, f"lstm_tuning_log_{WIN_KEY}.json")

# ---- Reproducibility ----
tf.random.set_seed(42)
np.random.seed(42)

# ---- GPU info (unchanged) ----
gpus = tf.config.list_physical_devices('GPU')
print("GPUs visible to TF:", gpus)
if gpus:
    try:
        tf.config.experimental.set_memory_growth(gpus[0], True)
    except Exception:
        pass
gpu_name = GPUtil.getGPUs()[0].name if GPUtil.getGPUs() else "None"
print("GPU in use:", gpu_name)

# ---- Safe loader: memory-mapped + window tag + NaN scan ----
def _pick_path(d, base, tag):
    p_tag = os.path.join(d, f"{base}_{tag}.npy")
    p_plain = os.path.join(d, f"{base}.npy")
    if os.path.exists(p_tag):   return p_tag
    if os.path.exists(p_plain): return p_plain
    raise FileNotFoundError(f"Missing {base}_{tag}.npy (or {base}.npy) in {d}")

def _nan_scan(mm, step=8192):
    n = mm.shape[0]
    for i in range(0, n, step):
        if np.isnan(mm[i:i+step]).any():
            raise ValueError("NaNs detected in array.")

def load_split_npys_memmap(d, tag):
    X_train = np.load(_pick_path(d,"X_train",tag), mmap_mode="r")
    y_train = np.load(_pick_path(d,"y_train",tag), mmap_mode="r")
    X_val   = np.load(_pick_path(d,"X_val",  tag), mmap_mode="r")
    y_val   = np.load(_pick_path(d,"y_val",  tag), mmap_mode="r")
    X_test  = np.load(_pick_path(d,"X_test", tag), mmap_mode="r")
    y_test  = np.load(_pick_path(d,"y_test", tag), mmap_mode="r")

    # Shape checks
    assert X_train.ndim==3 and X_val.ndim==3 and X_test.ndim==3, "X must be (N,T,F)"
    assert y_train.ndim==2 and y_val.ndim==2 and y_test.ndim==2, "y must be (N,12)"
    assert y_train.shape[1]==12 and y_val.shape[1]==12 and y_test.shape[1]==12, "y must have 12 steps"

    # Chunked NaN scans
    for arr in (X_train, X_val, X_test, y_train, y_val, y_test):
        _nan_scan(arr)

    return X_train, y_train, X_val, y_val, X_test, y_test

# ---- Load 96→12 ----
X_train, y_train, X_val, y_val, X_test, y_test = load_split_npys_memmap(DATA_DIR, WIN_TAG)
print(f"Train X: {X_train.shape} | y: {y_train.shape}")
print(f"Val   X: {X_val.shape}   | y: {y_val.shape}")
print(f"Test  X: {X_test.shape}  | y: {y_test.shape}")
assert X_train.shape[1] == 96, f"Expected T=96; got {X_train.shape[1]}"

timesteps  = X_train.shape[1]
n_features = X_train.shape[2]
out_steps  = y_train.shape[1]

# ---- 5% subset (materialize only the needed rows into RAM) ----
subset_frac = 0.05
n_train_sub = max(1, int(subset_frac * X_train.shape[0]))
n_val_sub   = max(1, int(subset_frac * X_val.shape[0]))

X_train_small = np.asarray(X_train[:n_train_sub])
y_train_small = np.asarray(y_train[:n_train_sub])
X_val_small   = np.asarray(X_val[:n_val_sub])
y_val_small   = np.asarray(y_val[:n_val_sub])

print(f"Subsets -> X_train_small {X_train_small.shape} | X_val_small {X_val_small.shape}")

# ---- Model builder (same as V1) ----
def make_builder(timesteps, n_features, out_steps):
    def build_model(hp):
        units   = hp.Int('units', min_value=32, max_value=128, step=32)
        dropout = hp.Float('dropout', min_value=0.0, max_value=0.5, step=0.1)
        lr      = hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])

        model = keras.Sequential([
            layers.Input(shape=(timesteps, n_features)),
            layers.LSTM(units),
            layers.Dropout(dropout),
            layers.Dense(out_steps)
        ])
        model.compile(
            optimizer=keras.optimizers.Adam(learning_rate=lr),
            loss='mse',
            metrics=['mae']
        )
        return model
    return build_model

builder = make_builder(timesteps, n_features, out_steps)

# ---- Tuner (same as V1) ----
tuner = kt.BayesianOptimization(
    builder,
    objective='val_loss',
    max_trials=20,
    directory=DATA_DIR,                    # keep artifacts next to data
    project_name=f'lstm_retune_{WIN_KEY}'  # unique per window
)

early_stop = keras.callbacks.EarlyStopping(
    monitor='val_loss', patience=4, restore_best_weights=True
)

# Slightly smaller batch for 96→12 to avoid GPU OOM (still comparable)
batch_size = 256  # use 512 if your GPU has headroom

# ---- Search ----
device = '/GPU:0' if tf.config.list_physical_devices('GPU') else '/CPU:0'
print("Using device:", device)

start_time = time.time()
with tf.device(device):
    tuner.search(
        X_train_small, y_train_small,
        validation_data=(X_val_small, y_val_small),
        epochs=50,
        batch_size=batch_size,
        callbacks=[early_stop],
        verbose=1
    )
end_time = time.time()

# ---- Save best HPs ----
best_hp = tuner.get_best_hyperparameters(1)[0]
hp_dict = {
    'units': best_hp.get('units'),
    'dropout': best_hp.get('dropout'),
    'learning_rate': best_hp.get('learning_rate')
}
with open(HP_PATH, 'w') as f:
    json.dump(hp_dict, f, indent=4)

# ---- Log (V1-style fields) ----
tmp_model = builder(best_hp)
total_params = tmp_model.count_params()
platform_info = platform.platform()

tune_log = {
    "window": WIN_KEY,
    "model": "LSTM",
    "task": "Smart Meter Energy Forecasting",
    "tuning_type": "BayesianOptimization",
    "subset_frac": subset_frac,
    "timesteps": int(timesteps),
    "n_features": int(n_features),
    "out_steps": int(out_steps),
    "tuning_time_minutes": round((end_time - start_time) / 60, 2),
    "best_hyperparameters": hp_dict,
    "total_params": int(total_params),
    "gpu_used": gpu_name,
    "platform": platform_info,
    "log_type": "Tuning"
}
with open(LOG_PATH, 'w') as f:
    json.dump(tune_log, f, indent=4)

print(f"\n[{WIN_KEY}] Best HPs saved → {HP_PATH}")
print(f"[{WIN_KEY}] Tuning log  → {LOG_PATH}")



Trial 20 Complete [00h 06m 46s]
val_loss: 0.09350237995386124

Best val_loss So Far: 0.09350237995386124
Total elapsed time: 01h 54m 57s

[96to12] Best HPs saved → /content/drive/MyDrive/EdgeMeter_AIv2/data/best_lstm_hp_96to12.json
[96to12] Tuning log  → /content/drive/MyDrive/EdgeMeter_AIv2/data/lstm_tuning_log_96to12.json


0

In [ ]:
import os
import time

# Delay a bit to ensure all file writes are complete
print("✅ All code executed. Disconnecting runtime in 5 seconds...")
time.sleep(60)

# Disconnect runtime
os.kill(os.getpid(), 9)


✅ All code executed. Disconnecting runtime in 5 seconds...
